# Día 4: Clases y el Principio de Responsabilidad Única (SRP)

## Descripción General

El **Principio de Responsabilidad Única (Single Responsibility Principle - SRP)** es el primero de los principios SOLID y establece que una clase debe tener una sola razón para cambiar. En otras palabras, cada clase debe tener una única responsabilidad bien definida.

Este principio es fundamental para crear código mantenible, testeable y flexible. Cuando una clase tiene múltiples responsabilidades, los cambios en una responsabilidad pueden afectar a las otras, creando acoplamiento y fragilidad.

**Conexión con Data/IA**: En proyectos de ML, es común ver clases que mezclan preparación de datos, entrenamiento, evaluación, y persistencia. Separar estas responsabilidades hace el código más testeable y reutilizable.

## Objetivos de Aprendizaje

Al finalizar este notebook, serás capaz de:

1. Comprender el Principio de Responsabilidad Única y su importancia
2. Identificar clases que violan el SRP
3. Refactorizar clases para que cumplan con el SRP
4. Diseñar clases cohesivas con una única responsabilidad
5. Aplicar el SRP en proyectos de IA y ciencia de datos
6. Desarrollar intuición para detectar múltiples responsabilidades

## 1. El Principio de Responsabilidad Única (SRP)

### 🎯 Contexto: Por Qué Importa

**Problema real en Data/IA**: 

Clase `ModelTrainer` hace: entrenar modelo, guardar checkpoints, enviar métricas a dashboard, generar reportes PDF, enviar emails. **5 responsabilidades** 💥 Cambiar formato de email = **modificar clase de training**. Bug en PDF = **training se rompe**. Testing imposible (mockear 5 cosas).

Con SRP: 5 clases separadas (`Trainer`, `CheckpointSaver`, `MetricsLogger`, `ReportGenerator`, `EmailSender`). Cambiar email = **modificar solo EmailSender** ✅ Bug en PDF = **training sigue funcionando** ✅

**Ejemplo concreto para juniors**:

Clase `User` hace: validar datos, guardar en DB, enviar email de bienvenida, generar token JWT, logging. **5 responsabilidades** 💥 Cambiar formato de email = **modificar clase User**. Testing = mockear DB + email + JWT + logging.

Con SRP: `User` (datos), `UserRepository` (DB), `EmailService` (email), `TokenService` (JWT), `Logger` (logging). **Testing simple** ✅ Cambios aislados ✅

**Consecuencias de NO usarlo**:
- **Cambios caros** → modificar una cosa rompe otras (efecto dominó)
- **Testing imposible** → mockear 5 dependencias por clase
- **Bugs multiplicados** → un bug afecta múltiples funcionalidades
- **Reusabilidad cero** → no puedes reusar una parte sin arrastrar todo
- **Merge conflicts** → 5 personas modifican la misma clase

### 📚 El Concepto

**Definición técnica**:

**SRP (Single Responsibility Principle)**: Una clase debe tener **una sola razón para cambiar**. Cada clase debe tener **una única responsabilidad** bien definida.

**Cómo funciona internamente**:
1. Identificas todas las razones por las que una clase podría cambiar
2. Si hay más de una razón → viola SRP
3. Separas cada responsabilidad en su propia clase
4. Cada clase ahora tiene una sola razón para cambiar
5. Clases colaboran mediante composición

**Terminología clave**:
- **Responsabilidad**: Razón para cambiar (feature, actor, caso de uso)
- **Cohesión**: Qué tan relacionadas están las cosas en una clase
- **Acoplamiento**: Qué tan dependientes son las clases entre sí
- **Razón para cambiar**: Cambio en requisitos que afecta la clase

**Reglas para identificar responsabilidades**:
1. **Pregunta "¿Por qué cambiaría?"** → cada respuesta = responsabilidad
2. **Cuenta los "y"** → "hace X y Y y Z" = 3 responsabilidades
3. **Identifica actores** → cada stakeholder = responsabilidad
4. **Busca verbos** → cada verbo principal = posible responsabilidad
5. **Mira imports** → muchos imports = muchas responsabilidades

### ❌ Ejemplo Incorrecto: Clase con Múltiples Responsabilidades

**Código**:

In [ ]:
# ❌ BAD: Class with multiple responsibilities
import json
import smtplib
from typing import List, Dict

class UserManager:
    """
    User manager - VIOLATES SRP!
    Has 4 responsibilities: validation, persistence, email, reporting.
    """
    
    def __init__(self, db_path: str, smtp_server: str) -> None:
        self.db_path = db_path
        self.smtp_server = smtp_server
        self.users: List[Dict] = []
    
    # Responsibility 1: Validation
    def validate_user(self, user: Dict) -> bool:
        """Validate user data."""
        if not user.get('email') or '@' not in user['email']:
            return False
        if not user.get('name') or len(user['name']) < 2:
            return False
        return True
    
    # Responsibility 2: Persistence (Database)
    def save_user(self, user: Dict) -> None:
        """Save user to database."""
        if not self.validate_user(user):
            raise ValueError("Invalid user data")
        
        # Save to file (simulating DB)
        with open(self.db_path, 'a') as f:
            f.write(json.dumps(user) + '\n')
        
        self.users.append(user)
        self.send_welcome_email(user)  # Coupled!
    
    # Responsibility 3: Email
    def send_welcome_email(self, user: Dict) -> None:
        """Send welcome email to user."""
        # Email logic (simulated)
        print(f"Sending email to {user['email']}...")
        # In reality: smtplib.SMTP(self.smtp_server)...
    
    # Responsibility 4: Reporting
    def generate_user_report(self) -> str:
        """Generate report of all users."""
        report = "User Report\n" + "=" * 50 + "\n"
        for user in self.users:
            report += f"Name: {user['name']}, Email: {user['email']}\n"
        return report

# Usage - tightly coupled
manager = UserManager("users.db", "smtp.example.com")
user = {"name": "Alice", "email": "alice@example.com"}
manager.save_user(user)
print(manager.generate_user_report())

**Problemas**:
- **4 responsabilidades** → validación, DB, email, reportes
- **4 razones para cambiar** → cambiar validación, DB, email, o reportes
- **Testing imposible** → mockear DB + SMTP + filesystem
- **Acoplamiento alto** → `save_user()` llama `send_welcome_email()`
- **Reusabilidad cero** → no puedes usar validación sin arrastrar DB/email
- **Merge conflicts** → 4 personas modifican la misma clase

### ✅ Ejemplo Correcto: Clases con Responsabilidad Única

**Código**:

In [ ]:
# ✅ GOOD: Each class has a single responsibility
import json
from typing import Dict, List
from dataclasses import dataclass

# Responsibility 1: Validation
class UserValidator:
    """
    Validates user data.
    Single responsibility: validation logic.
    """
    
    def validate(self, user: Dict) -> bool:
        """
        Validate user data.
        
        :param user: User dictionary
        :type user: Dict
        :return: True if valid
        :rtype: bool
        """
        if not user.get('email') or '@' not in user['email']:
            return False
        if not user.get('name') or len(user['name']) < 2:
            return False
        return True

# Responsibility 2: Persistence
class UserRepository:
    """
    Manages user persistence.
    Single responsibility: database operations.
    """
    
    def __init__(self, db_path: str) -> None:
        self.db_path = db_path
    
    def save(self, user: Dict) -> None:
        """
        Save user to database.
        
        :param user: User dictionary
        :type user: Dict
        """
        with open(self.db_path, 'a') as f:
            f.write(json.dumps(user) + '\n')
    
    def find_all(self) -> List[Dict]:
        """
        Load all users from database.
        
        :return: List of users
        :rtype: List[Dict]
        """
        try:
            with open(self.db_path, 'r') as f:
                return [json.loads(line) for line in f]
        except FileNotFoundError:
            return []

# Responsibility 3: Email
class EmailService:
    """
    Sends emails.
    Single responsibility: email delivery.
    """
    
    def __init__(self, smtp_server: str) -> None:
        self.smtp_server = smtp_server
    
    def send_welcome_email(self, user: Dict) -> None:
        """
        Send welcome email to user.
        
        :param user: User dictionary
        :type user: Dict
        """
        print(f"Sending email to {user['email']} via {self.smtp_server}...")
        # In reality: smtplib.SMTP(self.smtp_server)...

# Responsibility 4: Reporting
class UserReportGenerator:
    """
    Generates user reports.
    Single responsibility: report generation.
    """
    
    def generate(self, users: List[Dict]) -> str:
        """
        Generate report of users.
        
        :param users: List of users
        :type users: List[Dict]
        :return: Report string
        :rtype: str
        """
        report = "User Report\n" + "=" * 50 + "\n"
        for user in users:
            report += f"Name: {user['name']}, Email: {user['email']}\n"
        return report

# Coordinator (optional) - orchestrates but doesn't do the work
class UserService:
    """
    Coordinates user operations.
    Single responsibility: orchestration (not business logic).
    """
    
    def __init__(
        self,
        validator: UserValidator,
        repository: UserRepository,
        email_service: EmailService
    ) -> None:
        self.validator = validator
        self.repository = repository
        self.email_service = email_service
    
    def register_user(self, user: Dict) -> None:
        """
        Register a new user.
        
        :param user: User dictionary
        :type user: Dict
        :raises ValueError: If user data is invalid
        """
        if not self.validator.validate(user):
            raise ValueError("Invalid user data")
        
        self.repository.save(user)
        self.email_service.send_welcome_email(user)

# Usage - loosely coupled, easy to test
validator = UserValidator()
repository = UserRepository("users.db")
email_service = EmailService("smtp.example.com")
report_generator = UserReportGenerator()

service = UserService(validator, repository, email_service)
user = {"name": "Alice", "email": "alice@example.com"}
service.register_user(user)

# Generate report independently
users = repository.find_all()
print(report_generator.generate(users))

**Ventajas**:
- **1 responsabilidad por clase** → fácil de entender
- **1 razón para cambiar** → cambios aislados
- **Testing simple** → mockear una dependencia por test
- **Reusabilidad alta** → usar `UserValidator` en otros contextos
- **Sin merge conflicts** → cada persona modifica su clase
- **Composición flexible** → cambiar implementación sin afectar otras

### 📊 Comparación Lado a Lado

| Aspecto | Sin SRP | Con SRP |
|---------|---------|----------|
| Responsabilidades | 4 en 1 clase | 1 por clase |
| Razones para cambiar | 4 | 1 |
| Testing | Mockear 4 cosas | Mockear 1 cosa |
| Reusabilidad | Imposible | Alta |
| Merge conflicts | Frecuentes | Raros |
| Acoplamiento | Alto | Bajo |

### 💡 Aprendizaje Clave

**Puntos críticos a recordar**:
1. **Una razón para cambiar** → si hay 2+ razones, viola SRP
2. **Cohesión alta** → todo en la clase está relacionado
3. **Acoplamiento bajo** → clases independientes
4. **Testing simple** → mockear una cosa por test
5. **Reusabilidad** → usar partes sin arrastrar todo

**Cómo desarrollar intuición**:
- **Pregúntate**: "¿Por qué cambiaría esta clase?"
  - 1 razón → cumple SRP ✅
  - 2+ razones → viola SRP 💥

- **Pregúntate**: "¿Puedo describir la clase en una frase sin 'y'?"
  - SÍ → cumple SRP ✅
  - NO ("hace X y Y y Z") → viola SRP 💥

- **Pregúntate**: "¿Cuántos imports tiene?"
  - Pocos (2-3) → probablemente cumple SRP ✅
  - Muchos (10+) → probablemente viola SRP 💥

- **Pregúntate**: "¿Puedo testear esto fácilmente?"
  - SÍ (1-2 mocks) → cumple SRP ✅
  - NO (5+ mocks) → viola SRP 💥

**Cuándo usar / NO usar**:
- ✅ **Aplicar SRP cuando**:
  - Clase hace múltiples cosas no relacionadas
  - Testing es difícil (muchos mocks)
  - Cambios en una parte rompen otras
  - Múltiples personas modifican la misma clase
- ❌ **NO sobre-aplicar cuando**:
  - Clase es muy simple (< 50 líneas)
  - Responsabilidades están muy acopladas
  - Separar crea más complejidad que beneficio

**Patrones de refactorización**:
```python
# Antes: God Class
class ModelTrainer:
    def train()
    def save_checkpoint()
    def log_metrics()
    def send_email()

# Después: SRP
class Trainer:  # Solo training
class CheckpointSaver:  # Solo persistencia
class MetricsLogger:  # Solo logging
class EmailNotifier:  # Solo notificaciones
```

**Beneficios medibles**:
- **Testing +80% más rápido** → menos mocks
- **Bugs -60%** → cambios aislados
- **Merge conflicts -70%** → clases separadas

**Referencia oficial:** [SOLID Principles](https://en.wikipedia.org/wiki/SOLID)

## Resumen

1. **SRP** → una clase, una responsabilidad, una razón para cambiar
2. **Identificar violaciones** → cuenta razones para cambiar, busca "y" en descripción
3. **Refactorizar** → separar cada responsabilidad en su propia clase
4. **Beneficios** → testing simple, cambios aislados, reusabilidad alta
5. **Balance** → no sobre-aplicar en clases muy simples

## Preguntas de Autoevaluación

### 1. ¿Qué significa "una razón para cambiar"?

**Respuesta:** Una razón para cambiar es un cambio en requisitos que afectaría la clase. Por ejemplo, cambiar el formato de email, cambiar la base de datos, o cambiar la lógica de validación son razones diferentes.

### 2. ¿Cómo identificas si una clase viola el SRP?

**Respuesta:** Pregunta "¿por qué cambiaría esta clase?" Si hay más de una respuesta, viola SRP. También busca "y" en la descripción ("hace X y Y"), muchos imports, o testing difícil.

### 3. ¿Cuál es la diferencia entre cohesión y acoplamiento?

**Respuesta:** Cohesión es qué tan relacionadas están las cosas dentro de una clase (queremos alta). Acoplamiento es qué tan dependientes son las clases entre sí (queremos bajo).

### 4. ¿Por qué el SRP facilita el testing?

**Respuesta:** Con SRP, cada clase tiene una sola dependencia/responsabilidad, por lo que solo necesitas mockear una cosa por test. Sin SRP, necesitas mockear múltiples dependencias.

### 5. ¿Cuándo NO deberías aplicar SRP?

**Respuesta:** No sobre-apliques SRP en clases muy simples (< 50 líneas) o cuando separar responsabilidades crea más complejidad que beneficio. El objetivo es simplicidad, no pureza teórica.

## Recursos y Referencias Oficiales

### Documentación Oficial
- **[SOLID Principles](https://en.wikipedia.org/wiki/SOLID)**: Explicación de los 5 principios SOLID
- **[Clean Code by Robert C. Martin](https://www.oreilly.com/library/view/clean-code-a/9780136083238/)**: Libro fundamental sobre código limpio

### Mejores Prácticas
- **[Real Python - SOLID Principles](https://realpython.com/solid-principles-python/)**: Guía práctica de SOLID en Python

### Notas Importantes
- Todos los enlaces están actualizados a partir de 2025
- Se recomienda revisar la documentación oficial regularmente